In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
def remove_folder_contents(folder):
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                remove_folder_contents(file_path)
                os.rmdir(file_path)
        except Exception as e:
            print(e)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("secret_tokens")

!git clone https://github.com/kz-song/MMSRec.git

Cloning into 'MMSRec'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 82 (delta 14), reused 79 (delta 14), pack-reused 0
Unpacking objects: 100% (82/82), 18.82 MiB | 10.43 MiB/s, done.


In [3]:
%cd '/kaggle/working/MMSRec'
!sh weights/clip/download.sh
!pip install wandb easydict pynvml torchaudio decord jsonlines transformers torchvision

/kaggle/working/MMSRec
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-be9dlapk
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-be9dlapk
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 842.9 kB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369497 sha256=0034c6ddb4db9b0f8a01fdd7e6d743304df2e6e550adea2d624a006875e27679
  Stored in directory: /tmp/pip-ephem-wheel-cache-vx685a_z/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip
--2024-04-22 03:38:33--  https://openaipublic.azureedge.net/clip/models/40d365715913c9da98579312b702a82c18be219cc2a73407c4526f58eba950af/ViT-B-32.pt
Resolving openaipublic.azureedge.net (openaipublic.azureedge.net)... 13.107.246.38, 13.107.213.38, 2620:1ec:bdf::38,

In [4]:
%cd '/kaggle/working/MMSRec/dataset/amazon/raw'

# Amazon All Beauty
!mkdir Beauty
%cd ./Beauty
!t --no-check-certificate http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Beauty.csv
!wget --no-check-certificate http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/meta_Beauty.json.gz
%cd '/kaggle/working/MMSRec/dataset/amazon'

/kaggle/working/MMSRec/dataset/amazon/raw
/kaggle/working/MMSRec/dataset/amazon/raw/Beauty
/bin/bash: t: command not found
--2024-04-22 03:38:55--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/meta_Beauty.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99156148 (95M) [application/x-gzip]
Saving to: 'meta_Beauty.json.gz'

meta_Beauty.json.gz 100%[===================>]  94.56M  34.1MB/s    in 2.8s    

2024-04-22 03:38:58 (34.1 MB/s) - 'meta_Beauty.json.gz' saved [99156148/99156148]

/kaggle/working/MMSRec/dataset/amazon


In [5]:
!python process_item.py

python: can't open file '/kaggle/working/MMSRec/dataset/amazon/process_item.py': [Errno 2] No such file or directory


In [6]:
!python extract_features.py

python: can't open file '/kaggle/working/MMSRec/dataset/amazon/extract_features.py': [Errno 2] No such file or directory


In [7]:
!torchrun --nnodes=1:5 --nproc_per_node=1 --max_restarts=0 --rdzv_id=1 --rdzv_backend=c10d --rdzv_endpoint="127.0.0.1:12309" finetune_amazon.py

[2024-04-22 03:39:09,779] torch.distributed.run: [WARNING] master_addr is only used for static rdzv_backend and when rdzv_endpoint is not specified.
/opt/conda/bin/python3.10: can't open file '/kaggle/working/MMSRec/dataset/amazon/finetune_amazon.py': [Errno 2] No such file or directory
[2024-04-22 03:39:44,991] torch.distributed.elastic.multiprocessing.api: [ERROR] failed (exitcode: 2) local_rank: 0 (pid: 112) of binary: /opt/conda/bin/python3.10
Traceback (most recent call last):
  File "/opt/conda/bin/torchrun", line 8, in <module>
    sys.exit(main())
  File "/opt/conda/lib/python3.10/site-packages/torch/distributed/elastic/multiprocessing/errors/__init__.py", line 346, in wrapper
    return f(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/torch/distributed/run.py", line 806, in main
    run(args)
  File "/opt/conda/lib/python3.10/site-packages/torch/distributed/run.py", line 797, in run
    elastic_launch(
  File "/opt/conda/lib/python3.10/site-packages/torch/dis